In [7]:
bucket = "c56161a939430l3396553t1w744137092661-labbucket-rn642jaq01e9"
job_data_access_role = 'arn:aws:iam::744137092661:role/service-role/c56161a939430l3396553t1w7-ComprehendDataAccessRole-1P24MSS91ADHP'

### 1. Viewing the video files


In [8]:
!aws s3 ls s3://aws-tc-largeobjects/CUR-TF-200-ACMNLP-1/video/

2021-04-26 20:17:33  410925369 Mod01_Course Overview.mp4
2021-04-26 20:10:02   39576695 Mod02_Intro.mp4
2021-04-26 20:31:23  302994828 Mod02_Sect01.mp4
2021-04-26 20:17:33  416563881 Mod02_Sect02.mp4
2021-04-26 20:17:33  318685583 Mod02_Sect03.mp4
2021-04-26 20:17:33  255877251 Mod02_Sect04.mp4
2021-04-26 20:23:51   99988046 Mod02_Sect05.mp4
2021-04-26 20:24:54   50700224 Mod02_WrapUp.mp4
2021-04-26 20:26:27   60627667 Mod03_Intro.mp4
2021-04-26 20:26:28  272229844 Mod03_Sect01.mp4
2021-04-26 20:27:06  309127124 Mod03_Sect02_part1.mp4
2021-04-26 20:27:06  195635527 Mod03_Sect02_part2.mp4
2021-04-26 20:28:03  123924818 Mod03_Sect02_part3.mp4
2021-04-26 20:31:28  171681915 Mod03_Sect03_part1.mp4
2021-04-26 20:32:07  285200083 Mod03_Sect03_part2.mp4
2021-04-26 20:33:17  105470345 Mod03_Sect03_part3.mp4
2021-04-26 20:35:10  157185651 Mod03_Sect04_part1.mp4
2021-04-26 20:36:27  187435635 Mod03_Sect04_part2.mp4
2021-04-26 20:36:40  280720369 Mod03_Sect04_part3.mp4
2021-04-26 20:40:01  443479

In [9]:
pip install moviepy transformers accelerate torch

Note: you may need to restart the kernel to use updated packages.


### 2. Transcribing the videos


In [10]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline   
device_map = ""
if torch.cuda.is_available():
    device_map = "cuda:0"
else:
    device_map = "cpu"
torch_dtype = ""
if torch.cuda.is_available():
    torch_dtype = torch.float16
else:
    torch_dtype = torch.float32
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_version = "openai/whisper-large-v3"
whisper = AutoModelForSpeechSeq2Seq.from_pretrained( model_version, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
whisper.to(device_map)
processor = AutoProcessor.from_pretrained(model_version)
pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device_map,)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
import boto3
s3 = boto3.client('s3')
bucket_name = "aws-tc-largeobjects"
key  = "CUR-TF-200-ACMNLP-1/video/"
response = s3.list_objects_v2(Bucket="aws-tc-largeobjects", Prefix="CUR-TF-200-ACMNLP-1/video/")

In [12]:
import boto3
import os
from moviepy.editor import *
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
folder_path = 'text_files'
if os.path.exists(folder_path):
    pass
else:
    os.makedirs(folder_path)


s3 = boto3.client('s3')
bucket_name = "aws-tc-largeobjects"
key  = "CUR-TF-200-ACMNLP-1/video/"
response = s3.list_objects_v2(Bucket="aws-tc-largeobjects", Prefix="CUR-TF-200-ACMNLP-1/video/")
def convert_to_audio():
    for obj in response["Contents"]:
        file_name = obj["Key"].split("/")
        print(file_name)
        s3.download_file(bucket_name,obj["Key"],file_name[2])
        video_file = VideoFileClip(file_name[2])
        audio_file = video_file.audio
        file = file_name[2].split(".")
        print(file)
        audio_file.write_audiofile(f"{file[0]}.mp3")
        os.remove(file_name[2])
        video_file.close()
        audio_file.close()
convert_to_audio()

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5033:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2501:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_concat returned error: N

['CUR-TF-200-ACMNLP-1', 'video', 'Mod01_Course Overview.mp4']
['Mod01_Course Overview', 'mp4']
MoviePy - Writing audio in Mod01_Course Overview.mp3


MoviePy - Done.
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Intro.mp4']
['Mod02_Intro', 'mp4']
MoviePy - Writing audio in Mod02_Intro.mp3


MoviePy - Done.
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Sect01.mp4']
['Mod02_Sect01', 'mp4']
MoviePy - Writing audio in Mod02_Sect01.mp3


MoviePy - Done.
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Sect02.mp4']
['Mod02_Sect02', 'mp4']
MoviePy - Writing audio in Mod02_Sect02.mp3


...

MoviePy - Done.
['CUR-TF-200-ACMNLP-1', 'video', 'Mod06_Sect01.mp4']
['Mod06_Sect01', 'mp4']
MoviePy - Writing audio in Mod06_Sect01.mp3


MoviePy - Done.
['CUR-TF-200-ACMNLP-1', 'video', 'Mod06_Sect02.mp4']
['Mod06_Sect02', 'mp4']
MoviePy - Writing audio in Mod06_Sect02.mp3


MoviePy - Done.
['CUR-TF-200-ACMNLP-1', 'video', 'Mod06_WrapUp.mp4']
['Mod06_WrapUp', 'mp4']
MoviePy - Writing audio in Mod06_WrapUp.mp3


MoviePy - Done.
['CUR-TF-200-ACMNLP-1', 'video', 'Mod07_Sect01.mp4']
['Mod07_Sect01', 'mp4']
MoviePy - Writing audio in Mod07_Sect01.mp3


MoviePy - Done.


In [13]:
def convert_to_text():
    for obj in response["Contents"]:
        file_name = obj["Key"].split("/")
        file = file_name[2].split(".")
        audio_file = file[0]
        result = pipe(f"{audio_file}.mp3")
        with open(f"{folder_path}/{audio_file}.txt", 'w',encoding ="UTF-8") as file:
            file.write(result["text"])
    
convert_to_text()

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
--- Logging error ---
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all 

### 3. Normalizing the text


In [14]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [15]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
def normalize(data):
    filtered_sentence = []
    word = data.lower()
    word = re.sub('\s+', ' ',word)
    word = re.sub(r'[^\w\s]', '', word)
    words = word_tokenize(word)
    d = set(stopwords.words("english"))
    for text in words:
        if text not in d:
            filtered_sentence.append(text)
    sentence = " ".join(filtered_sentence)
    return sentence
def read_text():
    t = ""
    folder_path_text = "normalized_text"
    for obj in response["Contents"]:
        file_name = obj["Key"].split("/")
        file = file_name[2].split(".")
        text_file = f"text_files/{file[0]}.txt"
        print(file_name)
        normalize_file = f"normalized_text/{file[0]}.txt"
        with open(text_file,"r",encoding="UTF-8") as file:
            data  = file.read()
            text = normalize(data)
            
            if os.path.exists(f"{folder_path_text}"):
                pass
            else:
                os.makedirs(f"{folder_path_text}")
            with open(f"{normalize_file}","w") as file_2:
                file_2.write(text)
            t+=text
    return t
text = read_text()


['CUR-TF-200-ACMNLP-1', 'video', 'Mod01_Course Overview.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Intro.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Sect01.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Sect02.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Sect03.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Sect04.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_Sect05.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod02_WrapUp.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Intro.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect01.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect02_part1.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect02_part2.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect02_part3.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect03_part1.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect03_part2.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect03_part3.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect04_part1.mp4']
['CUR-TF-200-ACMNLP-1', 'video', 'Mod03_Sect04_par

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 4. Extracting key phrases and topics


In [16]:
pip install keybert

Note: you may need to restart the kernel to use updated packages.


In [17]:
from keybert import KeyBERT

In [18]:
key_bert_model = KeyBERT(model='all-mpnet-base-v2')

In [19]:
import json
for obj in response["Contents"]:
    file_name = obj["Key"].split("/")
    file = file_name[2].split(".")
    text_file = f"normalized_text/{file[0]}.txt"
    folder_name  = "transribed_files"
    with open(f"{text_file}","r") as file_2:
        data = file_2.read()
        output = key_bert_model.extract_keywords(data, 
                                         stop_words='english',
                                         keyphrase_ngram_range=(2, 3), 
                                         top_n=10,
                                         highlight=False,
                                         use_maxsum=True,
                                         nr_candidates =20,
                                         diversity=0.7)
        key_phrase= list(dict(output).keys())
        if os.path.exists(folder_name):
            pass
        else:
            os.makedirs(folder_name)
        with open(f"{folder_name}/{file[0]}.txt","w") as file_3:
            for data in key_phrase:
                file_3.write(data+"\n")
        


In [20]:
import nltk
from nltk.util import ngrams
from collections import Counter
for obj in response["Contents"]:
    file_name = obj["Key"].split("/")
    file = file_name[2].split(".")
    text_file = f"normalized_text/{file[0]}.txt"
    folder_name  = "transribed_files"
    with open(f"{text_file}","r") as file_2:
        text = file_2.read()
        tokens = nltk.word_tokenize(text)
        n=2
        top=5
        data_sequence = ngrams(tokens, n)
        data_frequency= Counter(data_sequence)
        data_gram = data_frequency.most_common(top)
        with open(f"{folder_name}/{file[0]}.txt","a") as file_3:
            for i in data_gram:
                d= i[0][0]+" "+i[0][1]
                
                file_3.write(d+"\n")
        

### 5. Creating the dashboard


In [21]:
import ipywidgets as widgets
from IPython.display import display, clear_output,Video,HTML
import os
bucket_name = "aws-tc-largeobjects"
key  = "CUR-TF-200-ACMNLP-1/video/"
text = ""
import time 
def find_file(keyword):
    files_name = []
    for transcribed_file in os.listdir("transribed_files"):
        if transcribed_file == "..ipynb_checkpoints":
            pass
        with open(os.path.join('transribed_files',transcribed_file), 'r') as file:
                if keyword in file.read():
                    files_name.append(transcribed_file)
    return files_name               
def fetch_video(file):
    url = f"https://{bucket_name}.s3.amazonaws.com/{key}{file}.mp4"
    return url
def load_video(file):
    url = fetch_video(file)
    time.sleep(1)
    display(Video(url,width=800))
    
def listbox_options_handler(option):
    options = None
    options = option['new']
    load_video(options)
def displaying_listbox(file):
    display(text_bar, search_button)
    list_box_widget = widgets.Select(
    options=file,
    description='Please Select the video')
    list_box_widget.observe(listbox_options_handler, names='value')
    display(list_box_widget)
    display(HTML("<p> Please double click on the options to view the video"))
def on_button_click(b):
    clear_output()
    text_search = text_bar.value
    text = text_bar.value
    file_list = find_file(text_search.lower())
    file_data = []
    
    if file_list:
        for file_name in file_list:
            temp = file_name.split(".")
            file_data.append(temp[0])
        displaying_listbox(file_data)
    else:
        print("Sorry no file found with string matching {}.".format(text_search))
    return file_list

text_bar = widgets.Text(placeholder='Enter text to search')
search_button = widgets.Button(description='Search',disabled=False)
search_button.on_click(on_button_click)
display(text_bar, search_button)



Text(value='', placeholder='Enter text to search')

Button(description='Search', style=ButtonStyle())